In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"crime-and-punishment.pdf")
pages = loader.load()

pages = pages[26:277]

text = ""

for page in pages:
    text += page.page_content
    
text = text.replace('\t', ' ')

In [5]:
from langchain import OpenAI
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

num_tokens = llm.get_num_tokens(text)

print (f"This book has {num_tokens} tokens in it")

c:\Users\2017e\miniconda3\envs\ragui\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


This book has 99932 tokens in it


In [7]:
# Loaders
from langchain.schema import Document

# Splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Model
from langchain.chat_models import ChatOpenAI

# Embedding Support
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Summarizer we'll use for Map Reduce
from langchain.chains.summarize import load_summarize_chain

# Data Science
import numpy as np
from sklearn.cluster import KMeans
openai_api_key=''

In [8]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=10000, chunk_overlap=3000)

docs = text_splitter.create_documents([text])

In [9]:
num_documents = len(docs)

print (f"Now our book is split up into {num_documents} documents")

Now our book is split up into 55 documents


In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

vectors = embeddings.embed_documents([x.page_content for x in docs])

c:\Users\2017e\miniconda3\envs\ragui\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
num_clusters = 11

kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(vectors)

In [12]:
kmeans.labels_


array([ 3,  3,  8,  8,  8,  2,  2,  2,  2,  2,  8,  5,  1,  6,  5,  5,  5,
        4,  4,  0,  0,  6,  6,  6,  6,  6, 10, 10,  5,  9,  6,  8,  8,  1,
        8,  1,  1,  1,  1,  1,  1,  1,  1,  1,  8,  7,  7,  1,  1,  8,  9,
        9,  9,  9,  1])

In [15]:

closest_indices = []

for i in range(num_clusters):
    
    distances = np.linalg.norm(vectors - kmeans.cluster_centers_[i], axis=1)
    
    closest_index = np.argmin(distances)
    
    closest_indices.append(closest_index)

In [16]:
selected_indices = sorted(closest_indices)
selected_indices

[0, 3, 8, 15, 17, 19, 23, 26, 39, 45, 51]

In [17]:
llm3 = ChatOpenAI(temperature=0,
                 openai_api_key=openai_api_key,
                 max_tokens=1000,
                 model='gpt-3.5-turbo'
                )

c:\Users\2017e\miniconda3\envs\ragui\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [19]:
from langchain_core.prompts import PromptTemplate

map_prompt = """
You will be given a single passage of a book. This section will be enclosed in triple backticks (```)
Your goal is to give a summary of this section so that a reader will have a full understanding of what happened.
Your response should be at least three paragraphs and fully encompass what was said in the passage.

```{text}```
FULL SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

In [20]:
map_chain = load_summarize_chain(llm=llm3,
                             chain_type="stuff",
                             prompt=map_prompt_template)

In [21]:
selected_docs = [docs[doc] for doc in selected_indices]


In [22]:
summary_list = []

for i, doc in enumerate(selected_docs):
    
    chunk_summary = map_chain.run([doc])
    
    summary_list.append(chunk_summary)
    
    print (f"Summary #{i} (chunk #{selected_indices[i]}) - Preview: {chunk_summary[:250]} \n")

c:\Users\2017e\miniconda3\envs\ragui\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Summary #0 (chunk #0) - Preview: In this passage from Crime and Punishment, Marmeladov recounts the tragic story of his wife, Katerina Ivanovna, and their struggles with poverty and hardship. After her first husband's death, Katerina was left with three children in a remote district 

Summary #1 (chunk #3) - Preview: In this passage from "Crime and Punishment," we are introduced to the chaotic and impoverished living conditions of the Marmeladov family. The room they occupy is cramped and dingy, with a sofa covered in holes and a kitchen table that is unpainted a 

Summary #2 (chunk #8) - Preview: In this passage, Raskolnikov receives a letter from his mother, Pulcheria Raskolnikov, and his sister, Dounia. The letter discusses their plans to travel to meet him, as well as Dounia's upcoming marriage to Mr. Luzhin. Pulcheria expresses her love f 

Summary #3 (chunk #15) - Preview: In this passage from Crime and Punishment, the protagonist, Raskolnikov, experiences a moment of clarity and

In [23]:
summaries = "\n".join(summary_list)

# Convert it back to a document
summaries = Document(page_content=summaries)

print (f"Your total summary has {llm.get_num_tokens(summaries.page_content)} tokens")

Your total summary has 3379 tokens


In [114]:
llm4 = ChatOpenAI(temperature=0,
                 openai_api_key=openai_api_key,
                 max_tokens=4500,
                 model='gpt-4',
                 request_timeout=120
                )

In [116]:

combine_prompt = """
You will be given a series of summaries from a book. The summaries will be enclosed in triple backticks (```)
The reader should be able to grasp what happened in the book.
Summery must be of 5000 words.

```{text}```
SUMMARY:
"""



combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [117]:
reduce_chain = load_summarize_chain(llm=llm4,
                             chain_type="stuff",
                             prompt=combine_prompt_template,
                                   )

In [121]:
output = reduce_chain.run([summaries])


In [122]:
print (output)


"Crime and Punishment" is a novel that explores the themes of poverty, sacrifice, and resilience in the face of adversity. The story revolves around the lives of several characters, including Marmeladov, his wife Katerina Ivanovna, their daughter Sonia, and Raskolnikov, a young man grappling with his own moral dilemmas.

Marmeladov recounts his family's struggles with poverty and hardship, which have led to his daughter Sonia turning to prostitution to support them. Despite their dire circumstances, Marmeladov expresses gratitude for the sacrifices made by his wife and daughter. The family's living conditions are depicted as chaotic and impoverished, with Katerina Ivanovna often in a state of feverish emotion due to their desperate situation.

Raskolnikov, on the other hand, lives in isolation, consumed by his own thoughts and neglecting his basic needs. He receives a letter from his mother and sister, revealing their plans to visit him and his sister's upcoming marriage to Mr. Luzhin.

In [124]:
from fpdf import FPDF

def text_to_pdf(text, output_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    pdf.set_left_margin(10)
    pdf.set_right_margin(10)
    
    pdf.multi_cell(0, 10, txt=text)
    
    pdf.output(output_path)




output_pdf_path = "output.pdf"
text_to_pdf(output, output_pdf_path)
print(f"PDF saved to {output_pdf_path}")


PDF saved to output.pdf
